In [3]:
## Importing libraries: PySimpleGUI is used for generating GUI for user input and PuLP is used for implementing linear programming 
import PySimpleGUI as sg
from pulp import Amply
from pulp import *


## Create Solver function:

def Solve_3_in_a_Row():
    
    ## GUI generating part:
    
    layout = [[ sg.Text('Please enter the grid size'),],
          [sg.Input(do_not_clear=True)],
          [sg.Text('', key='_OUTPUT_')],
          [sg.Button('Ok')]]

    win1 = sg.Window('Grid size selection', layout)
    win2_active=False
    while True:
        ev1, vals1 = win1.Read(timeout=100)
    
        if ev1 is None:
            break
    
    
        if ev1 == 'Ok'  and not win2_active:
            win2_active = True
            win1.Close()
            n=int(vals1[0])
            layout2=[]
            for i in range(n):
                layout2.append([])
                for j in range(n):
                    layout2[i].append(sg.InputText(size=(3,1)))
            
            layout2.append([sg.Button('Ok'), sg.Button('Cancel')])
            layout2.insert(0,[sg.Text('3 in a row initial grid:'+'\n'+'Please follow below stated instructions'+'\n'+' (1) Only enter capital letters'+'\n'+' (2) Use B for blue and W for white'+'\n'+' (3) Leave the unfilled cells blank')])
            win2 = sg.Window('Grid Layout', layout2)
            while True:
                ev2, vals2 = win2.read()
            
                if ev2 in (None, 'Cancel','Ok'):
                    
                    win2_active = False
                    win2.Close()
                    break
        
        
            break
    
    ## Mapping strings with corresponding integer
    
    b_map={'': 0, 'B': 1, 'W':0}
    w_map={'': 0, 'W': 1, 'B':0}
    b={}
    w={}
    k=0
    
    ## Create parameters b(i,j) and w(i,j):
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            b[i,j]=b_map[vals2[k]]
            w[i,j]=w_map[vals2[k]]
            k=k+1
    ## Index range selection:
    
    I=range(1,n+1)
    J=range(1,n+1)
    
    ## Creating a LP problem using pulp's LpProblem:
    
    prob = LpProblem("3-in-a-Row Problem",LpMinimize)

    ## Decision Variables:
    x=LpVariable.dicts("X",(I,J),0,1,cat='Binary')
    y=LpVariable.dicts("y",(I,J),0,1,cat='Binary')

    ## Objective Function:
    prob+=0,"Arbitrary Objective Function"

    ## Constraints:
    
    ## Constraint 1:
    for i in I:
        for j in J:
            prob += x[i][j] + y[i][j] ==1 ,""

    ## Constraint 2:
    for i in range(1,n-1):
        for j in J:
            prob += x[i][j] + x[i+1][j] + x[i+2][j] <= 2 , ""

    ## Constraint 3:
    for i in range(1,n-1):
        for j in J:
            prob += y[i][j] + y[i+1][j] + y[i+2][j] <= 2 , ""

    ## Constraint 4:        
    for i in I:
        for j in range(1,n-1):
            prob += x[i][j] + x[i][j+1] + x[i][j+2] <= 2 , ""

    ## Constraint 5:
    for i in I:
        for j in range(1,n-1):
            prob += y[i][j] + y[i][j+1] + y[i][j+2] <= 2 , ""

    ## Constraint 6:        
    for i in I:
        for j in J:
            prob += w[i,j] <= x[i][j] , ""

    ## Constraint 7:
    for i in I:
        for j in J:
            prob += b[i,j] <= y[i][j] , ""

    ## Constraint 8:
    for j in J:
        prob += lpSum([x[i][j] for i in I]) == n/2 , ""
        
    ## Constraint 9:
    for i in I:
        prob += lpSum([x[i][j] for j in J]) == n/2 , ""

    ## Constraint 10:    
    for j in J:
        prob += lpSum([y[i][j] for i in I]) == n/2 , ""
    
    ## Constraint 11:
    for i in I:
        prob += lpSum([y[i][j] for j in J]) == n/2 , ""
    
    ## Solving LP problem:
    prob.solve()
     
    ## Solution Display:
    Solution=[]
    for i in range(1,n+1):
        for j in range(1,n+1):
            if x[i][j].varValue==1:
                Solution.append((i,j,'W'))
            else:
                Solution.append((i,j,'B'))
    
    k=0
    N=n*n
    for i in range(0,N):
        print(Solution[i][2],end=" ")
        k=k+1
        if k%n==0:
            print("\n")





Solve_3_in_a_Row()


W B B W W B 

B W W B B W 

B B W W B W 

W W B B W B 

B B W B W W 

W W B W B B 

